Voy a dejar un poco más limpio el dataframe. Eliminaré variables que no aportan mucho (porque ya tienen un one-hot encoding, por ejemplo) y luego limpiaré aquellas observaciones que tienen `n_baths`=0.

In [1]:
import pandas as pd

In [2]:
df=pd.read_excel('data/rmbsinbcn_marzo2024.xlsx')

Elimino `terrace_m2` porque no todos los pisos tienen información sobre su metraje. En cambio, sí sobre si tienen o no una terraza.

In [3]:
df=df.drop(['year_cons','floor','consum_EPC',
            'kwhm2_year','emission_EPC','kgCO2m2_year',
            'terrace_m2'],
            axis=1)

In [4]:
df = df[df['n_baths'] != 0]
len(df)

1646

Ahora elimino duplicados.

In [5]:
columns_to_keep = ['codigo_inmueble', 'Lon_X', 'Lat_Y']

# Crear una lista de columnas excluyendo las columnas que deseas mantener
columns_to_check_duplication = [col for col in df.columns if col not in columns_to_keep]

# Si quieres modificar directamente el DataFrame original, puedes hacer:
df.drop_duplicates(subset=columns_to_check_duplication, keep='first', inplace=True)

# Verificar el DataFrame resultante
len(df)

1595

In [6]:
df.to_excel('data/filtrado/rmbsinbcn_marzo2024-f1.xlsx', index=False)